In [ ]:
#dbutils.widgets.text("dias_atras", defaultValue="")
#dias_atras = dbutils.widgets.get("dias_atras")

In [1]:
#!pip install pandas-gbq

In [3]:
import pandas_gbq
import numpy as np
import requests
import datetime
import pandas as pd
import re
import json
from datetime import date, timedelta
from datetime import datetime
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr,first, last,when, split, col,lit, concat, date_format,from_utc_timestamp,to_utc_timestamp,to_timestamp, regexp_replace,concat_ws
from google.cloud import bigquery
client = bigquery.Client(location="us-central1")
#print("Client creating using default project: {}".format(client.project))
pd.set_option('display.width', 1000)
pd.set_option("max_colwidth",10000)
pd.set_option("max_rows",1000)
pd.set_option("max_columns",100)

In [4]:
spark = SparkSession.builder.appName("SanboxVT1_YT1").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/04 20:51:42 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/12/04 20:51:42 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/12/04 20:51:42 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/12/04 20:51:42 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [5]:
current_date = datetime.today()
dias_atras = 1
#dias_atras = int(dias_atras)
str_day = (current_date - timedelta(days = dias_atras)).strftime("%Y-%m-%d")
print(str_day)

2023-12-03


## YTS1 Metadata

In [6]:
#df_yts1_metadata = spark.read.format('bigquery').option('project','saas-analytics-io').option('table','yts1_v1data1.metadata').option("filter", """date(time) = '%s'"""%(str_day)).load()
#df_yts1_metadata = spark.read.format('bigquery').option('project','saas-analytics-io')\
#.option('table','yts1_v1data1.metadata').option("filter", """date(time) between "2023-11-01" and "2023-11-30" """).load()

df_yts1_metadata = spark.read.format('bigquery').option('project','staging-mojixretail-io')\
.option('table','silver.kafka_metadata').option("filter", """date(time) between "2023-11-01" and "2023-11-30" """).load()


In [8]:
print("Total Registros metadata ", df_yts1_metadata.count())

Total Registros metadata  1396874


## VTS1 Metadata

In [9]:
#df_vts1_metadata = spark.read.format('bigquery').option('project','saas-analytics-io').option('table','vts1_v1data1.metadata').option("filter", """date(time) = '%s'"""%(str_day)).load()
#df_vts1_metadata = spark.read.format('bigquery').option('project','saas-analytics-io')\
#.option('table','vts1_v1data1.metadata').option("filter", """date(time) between "2023-11-01" and "2023-11-30" """).load()


df_vts1_metadata = spark.read.format('bigquery').option('project','saas-sb-mojixtrack-io-293717')\
.option('table','silver.kafka_metadata').option("filter", """date(time) between "2023-11-01" and "2023-11-30" """).load()

In [10]:
print("Total Registros metadata ", df_vts1_metadata.count())

Total Registros metadata  4021644


### YTS1 Data Prepairing and groupping

In [8]:
df_yts1_metadata = df_yts1_metadata.withColumn('datetime_h', date_format(df_yts1_metadata['time'], "d/M/y H"))
df_yts1_metadata = df_yts1_metadata.withColumn('datetime_h', to_utc_timestamp(to_timestamp(df_yts1_metadata['datetime_h'],'d/M/y H'), 'UTC'))
df_yts1_metadata = df_yts1_metadata.withColumn('datetime_h_LA', from_utc_timestamp(col("datetime_h"),"America/Los_Angeles"))


df_yts1_metadata_agg = df_yts1_metadata.groupBy(['datetime_h_LA','datetime_h','tenantCode','thingType']).count()
df_yts1_metadata_agg = df_yts1_metadata_agg.withColumnRenamed("count","mojix_blink_count")


In [9]:
df_yts1_metadata_agg.show()

+-------------------+-------------------+-------------+-------------+-----------------+
|      datetime_h_LA|         datetime_h|   tenantCode|    thingType|mojix_blink_count|
+-------------------+-------------------+-------------+-------------+-----------------+
|2023-11-06 08:00:00|2023-11-06 16:00:00|         CENT|   EPCISEVENT|                4|
|2023-11-17 10:00:00|2023-11-17 18:00:00|       JFRESH|  DISPOSITION|                5|
|2023-11-17 10:00:00|2023-11-17 18:00:00|          PEI|  DISPOSITION|                5|
|2023-11-17 19:00:00|2023-11-18 03:00:00|         FALA|      CONTENT|               26|
|2023-11-17 17:00:00|2023-11-18 01:00:00|        FISIA|REFERENCELIST|                6|
|2023-11-20 10:00:00|2023-11-20 18:00:00|       CELINE|REFERENCELIST|                4|
|2023-11-22 18:00:00|2023-11-23 02:00:00|         CHIP|         ITEM|              762|
|2023-11-28 17:00:00|2023-11-29 01:00:00|         CHIP|   EPCISEVENT|              737|
|2023-11-30 01:00:00|2023-11-30 

### VTS1 Data Prepairing and groupping

In [10]:
df_vts1_metadata = df_vts1_metadata.withColumn('datetime_h', date_format(df_vts1_metadata['time'], "d/M/y H"))
df_vts1_metadata = df_vts1_metadata.withColumn('datetime_h', to_utc_timestamp(to_timestamp(df_vts1_metadata['datetime_h'],'d/M/y H'), 'UTC'))
df_vts1_metadata = df_vts1_metadata.withColumn('datetime_h_LA', from_utc_timestamp(col("datetime_h"),"America/Los_Angeles"))

df_vts1_metadata_agg = df_vts1_metadata.groupBy(['datetime_h_LA','datetime_h','tenantCode','thingType']).count()
df_vts1_metadata_agg = df_vts1_metadata_agg.withColumnRenamed("count","mojix_blink_count")


In [11]:
df_vts1_metadata_agg.show()

+-------------------+-------------------+----------+---------+-----------------+
|      datetime_h_LA|         datetime_h|tenantCode|thingType|mojix_blink_count|
+-------------------+-------------------+----------+---------+-----------------+
|2023-11-07 12:00:00|2023-11-07 20:00:00|     HONDA|   DEFECT|            11220|
|2023-11-16 01:00:00|2023-11-16 09:00:00|     HONDA|  VEHICLE|               73|
|2023-11-17 21:00:00|2023-11-18 05:00:00|     HONDA|   DEFECT|             6120|
|2023-11-10 09:00:00|2023-11-10 17:00:00|     HONDA| TRIGGERS|              121|
|2023-11-13 01:00:00|2023-11-13 09:00:00|     HONDA| TRIGGERS|               71|
|2023-11-07 09:00:00|2023-11-07 17:00:00|     HONDA|   DEFECT|            11251|
|2023-11-10 11:00:00|2023-11-10 19:00:00|     HONDA|   DEFECT|             8460|
|2023-11-13 10:00:00|2023-11-13 18:00:00|     HONDA| TRIGGERS|               58|
|2023-11-17 00:00:00|2023-11-17 08:00:00|     HONDA|   DEFECT|             6120|
|2023-10-31 22:00:00|2023-11

### Write data in Bigquery

In [12]:
bucket = "finops-outputs"
spark.conf.set('temporaryGcsBucket', bucket)

In [13]:
print("Write YTS1 data in Big Query Tables")

df_yts1_metadata_agg.write.format('bigquery') \
    .option('table', 'saas-analytics-io.processed.yts1_event_finops') \
    .mode('append') \
    .save()



Write YTS1 data in Big Query Tables


In [14]:
print("Write VTS1 data in Big Query Tables")

df_vts1_metadata_agg.write.format('bigquery') \
    .option('table', 'saas-analytics-io.processed.vts1_event_finops') \
    .mode('append') \
    .save()

Write VTS1 data in Big Query Tables
